In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision import datasets
from torchvision.transforms import v2 as transforms
from torchvision.ops import Conv2dNormActivation

from dataclasses import dataclass
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sn

import matplotlib.pyplot as plt
import time
import numpy as np
import random
import warnings
import os
from tqdm import tqdm

import pandas as pd

%matplotlib inline
warnings.filterwarnings("ignore")

In [3]:
#Set seed for reproducibilty
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
       torch.cuda.manual_seed(seed)
       torch.cuda.manual_seed_all(seed)
       torch.backends.cudnn.deterministic = True
       torch.backends.cudnn.benchmark = True

set_seed(42)

In [2]:
import requests

# The URL for the file you want to download
url = "https://www.dropbox.com/s/45jdd8padeyjq6t/10_Monkey_Species.zip?dl=1"
# The local filename to save the file as
output_filename = "10_Monkey_Species.zip"

print(f"Downloading {output_filename} from {url}...")

# Send a GET request to the URL
response = requests.get(url, stream=True)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the output file in write-binary mode
    with open(output_filename, "wb") as f:
        # Write the content of the response to the file in chunks
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Download completed successfully.")
else:
    print(f"Failed to download file. Status code: {response.status_code}")



Download completed successfully.


In [ ]:
import zipfile
import os

# --- Configuration ---
zip_file_path = '10_Monkey_Species.zip'
extract_to_directory = 'monkey_species_data'

# --- Main Logic ---
# First, check if the zip file actually exists
if not os.path.exists(zip_file_path):
    print(f"Error: The file '{zip_file_path}' was not found.")
    print("Please make sure you have downloaded the file first.")
else:
    # Create the extraction directory if it doesn't exist.
    # The exist_ok=True argument prevents an error if the directory is already there.
    os.makedirs(extract_to_directory, exist_ok=True)
    
    print(f"Extracting '{zip_file_path}' into '{extract_to_directory}'...")
    
    try:
        # Use a 'with' statement to safely open and automatically close the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # extractall() extracts all members from the archive into the specified directory
            zip_ref.extractall(extract_to_directory)
        
        print("\nExtraction complete!")
        print(f"Data is available in the directory: '{os.path.abspath(extract_to_directory)}'")

    except zipfile.BadZipFile:
        print(f"Error: The file '{zip_file_path}' is not a valid zip file or it is corrupted.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")



In [ ]:
@dataclass(frozen=True)
class TrainingConfig:
      ''' Configuration for Training '''
      batch_size: int = 32
      num_epochs: int = 100
      learning_rate: float = 1e-4

      log_interval: int = 1
      test_interval: int = 1
      data_root: int = "./"
      num_workers: int = 5
      device: str = "cuda"

train_config = TrainingConfig()
DEVICE = torch.device("cuda") if torch.cuda.is_available() else "cpu"
print("Available Device: ", DEVICE)